In [ ]:
# Import necessary libraries
from pyspark.sql import SparkSession

# Create a Spark session
spark = SparkSession.builder.appName("GenesysETL").getOrCreate()

# Replace 'jdbc_url', 'user', 'password' with your Genesys database connection details
genesys_df = spark.read.format("jdbc").option("url", "jdbc_url").option("dbtable", "genesis.table_name").option("user", "user").option("password", "password").load()

In [ ]:
# Use Databricks SQL syntax to read the configuration table from the Databricks catalog
config_df = spark.sql("SELECT * FROM administrative.bronze.table_configuration WHERE schema_name = 'genesys'")

In [ ]:
# Filter the configuration DataFrame where active field is true
active_tables_df = config_df.filter(config_df["active"] == True)


In [ ]:
# Iterate over the active tables and save them to bronze.genesys.table_name
for row in active_tables_df.collect():
    table_name = row["table_name"]
    genesys_table_df = genesys_df.filter(genesys_df["table_name"] == table_name)
    genesys_table_df.write.format("parquet").mode("overwrite").saveAsTable("bronze.genesys." + table_name)
